# Storstockholms Lokaltrafik network optimization case -- Shortest path

Storstockholms Lokaltrafik has a fault in its network at node 8. It has a technician ready to be dispatched from either node1 or node 2. The nodes denote the points in the network, and the arcs are the routes between the points. Travel times in minutes are indicated next to each arc. 

<img style="float: center;" width="300" height="300" src="sl_map.png">

This case study will formulate a network model that simultaneously minimises the time it takes to put a technician at node 8 and determines the optimal route taken by the technician.

In [1]:
# packages
import pyomo.environ as pyo
import pandas as pd

In [2]:
#Preparing data input
# Add a dummy node as the starting point. Supose the dummy node is node 0
RawData=[
    [0, 1, 0, 0],
    [0, 2, 0, 1],
    [1, 3, 40, 0],
    [1, 4, 50, 0],
    [2, 4, 30, 1],
    [2, 5, 40, 0],
    [3, 4, 20, 0],
    [3, 7, 55, 0],
    [3, 8, 60, 0],
    [4, 7, 30, 0],
    [4, 9, 20, 1],
    [5, 4, 30, 0],
    [5, 8, 30, 0],
    [5, 9, 20, 0],
    [7, 8, 10, 0],
    [9, 8, 15, 1]
]
column=["index_i", "index_j", "cost", "X_flow"]

#For convenience creating dataframe with all the inputs
DataDf=pd.DataFrame(RawData, columns=column)
DataDf.head()

,index_i,index_j,cost,X_flow
0,0,1,0,0
1,0,2,0,1
2,1,3,40,0
3,1,4,50,0
4,2,4,30,1


In [7]:
#Initializing pyomo model
model = pyo.ConcreteModel()

#Defining sets for receiving nodes and for sending nodes
model.From=pyo.Set(initialize=range(0,10))
model.To=pyo.Set(initialize=range(0,10))

#Defining variables
model.X=pyo.Var(model.From,model.To,domain=pyo.NonNegativeReals)

#Defining constraints
model.constraints = pyo.ConstraintList()

#Adding flow-Conservation Constraints
model.constraints.add(model.X[0,1]+model.X[0,2] == 1)
model.constraints.add(model.X[1,3]+model.X[1,4] == model.X[0,1])
model.constraints.add(model.X[2,4]+model.X[2,5] == model.X[0,2])
model.constraints.add(model.X[3,4]+model.X[3,7]+model.X[3,8] == model.X[1,3])
model.constraints.add(model.X[1,4]+model.X[2,4]+model.X[3,4]+model.X[5,4] == model.X[4,7]+model.X[4,9])
model.constraints.add(model.X[4,7]+model.X[3,7] == model.X[7,8])
model.constraints.add(model.X[4,9]+model.X[5,9] == model.X[9,8])
model.constraints.add(model.X[3,8]+model.X[7,8]+model.X[5,8]+model.X[9,8] == 1)

#Defining objective function
def obj_rule(model):
    return sum(model.X[DataDf.iloc[row]["index_i"],
                       DataDf.iloc[row]["index_j"]]*DataDf.iloc[row]["cost"] 
                       for row in range(len(DataDf)) )

# minize the cost/time
model.OBJ = pyo.Objective(rule=obj_rule,sense=pyo.minimize)

In [8]:
import sys
solvername='glpk'
solverpath_folder= r'C:\Users\rantao\Anaconda3\envs\SecEnvironment\Lib\site-packages\pyomo\solvers\plugins\solvers\GLPK.py'
solverpath_exe=r'C:\Users\rantao\Anaconda3\envs\SecEnvironment\Library\bin\glpsol.exe'

sys.path.append(solverpath_folder)

#Choosing solver
# solver = pyo.SolverFactory(solvername)
solver = pyo.SolverFactory(solvername, executable=solverpath_exe)

#Solving the model
sol = solver.solve(model)

#Checking the results
# total travel time is 65
print(sol)


Problem: 
- Name: unknown
  Lower bound: 65.0
  Upper bound: 65.0
  Number of objectives: 1
  Number of constraints: 9
  Number of variables: 17
  Number of nonzeros: 29
  Sense: minimize
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 0
      Number of created subproblems: 0
  Error rc: 0
  Time: 0.06699752807617188
Solution: 
- number of solutions: 0
  number of solutions displayed: 0



In [11]:
# model.OBJ.display()

# show the route
model.pprint()

4 Set Declarations
    From : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :   10 : {0, 1, 2, 3, 4, 5, 6, 7, 8, 9}
    To : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :   10 : {0, 1, 2, 3, 4, 5, 6, 7, 8, 9}
    X_index : Size=1, Index=None, Ordered=True
        Key  : Dimen : Domain  : Size : Members
        None :     2 : From*To :  100 : {(0, 0), (0, 1), (0, 2), (0, 3), (0, 4), (0, 5), (0, 6), (0, 7), (0, 8), (0, 9), (1, 0), (1, 1), (1, 2), (1, 3), (1, 4), (1, 5), (1, 6), (1, 7), (1, 8), (1, 9), (2, 0), (2, 1), (2, 2), (2, 3), (2, 4), (2, 5), (2, 6), (2, 7), (2, 8), (2, 9), (3, 0), (3, 1), (3, 2), (3, 3), (3, 4), (3, 5), (3, 6), (3, 7), (3, 8), (3, 9), (4, 0), (4, 1), (4, 2), (4, 3), (4, 4), (4, 5), (4, 6), (4, 7), (4, 8), (4, 9), (5, 0), (5, 1), (5, 2), (5, 3), (5, 4), (5, 5), (5, 6), (5, 7), (5, 8), (5, 9), (6, 0), (6, 1), (6, 2), (6, 3), (6,